# 案例-手机价格预测

In [8]:
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torchsummary import summary

# 1.获取数据
# 1.1读取数据
data = pd.read_csv(r"F:\Maker\Learn_Systematically\6_Deep_learning\2_neural_network\手机价格预测.csv")
x = data.iloc[:, :-1]
y = data.iloc[:, -1]

# 1.2划分数据集
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

x_train = torch.tensor(x_train.values,dtype = torch.float32)
x_test = torch.tensor(x_test.values, dtype = torch.float32)
y_train = torch.tensor(y_train.values, dtype = torch.int64)
y_test = torch.tensor(y_test.values, dtype = torch.int64)

# 1.3封装Tensor
train_dataset = TensorDataset(x_train, y_train)
test_dataset = TensorDataset(x_test, y_test)

# 1.4构建数据迭代器
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle= True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle = False)

# 2.构建模型
# 2.1类
class model(nn.Module):
# 2.2 init方法
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(in_features=20, out_features=64)
        self.layer2 = nn.Linear(in_features=64, out_features=128)
        self.layer3 = nn.Linear(in_features=128, out_features=4)
        self.dropout = nn.Dropout(p = 0.9)

    # forward方法
    def forward(self, x):
        x = self.layer1(x)
        x = self.dropout(x)
        x = torch.relu(x)
        x = self.layer2(x)
        x = self.dropout(x)
        x = torch.relu(x)
        out = self.layer3(x)
        return out

# 3.模型训练
def train():
    phone_model = model()
    # 损失
    cri = nn.CrossEntropyLoss()
    # 优化器
    optimizer = torch.optim.SGD(phone_model.parameters(), lr = 0.01)
    # 遍历
    epochs = 20
    for epoch in range(epochs):
        loss_sum = 0
        sample = 0.1
        for x, y in train_dataloader:
            y_pre = phone_model(x)
            loss = cri(y_pre, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_sum += loss.item()
            sample += 1
        print(loss_sum / sample)  # 出现了nan，表示梯度爆炸
    torch.save(phone_model.state_dict(), r'F:\Maker\Learn_Systematically\6_Deep_learning\2_neural_network\phone.pth')
# 4.模型预测
def test():
    my_model = model()
    my_model.load_state_dict(torch.load(r'F:\Maker\Learn_Systematically\6_Deep_learning\2_neural_network\phone.pth'))

    correct = 0
    for x, y in test_dataloader:
        y_pre = my_model(x)
        y_index = torch.argmax(y_pre, dim = 1)
        correct += (y_index == y).sum()
    acc = correct.item() / len(test_dataset)
    print("准确率：",acc)


if __name__ == '__main__':
    my_model = model()
    summary(my_model, input_size=(20, ), batch_size=10)
    train()
    test()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [10, 64]           1,344
           Dropout-2                   [10, 64]               0
            Linear-3                  [10, 128]           8,320
           Dropout-4                  [10, 128]               0
            Linear-5                    [10, 4]             516
Total params: 10,180
Trainable params: 10,180
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.03
Params size (MB): 0.04
Estimated Total Size (MB): 0.07
----------------------------------------------------------------
1538.2987212527103
7.629540157818544
1.3887939811765642
1.3872321863760655
1.3865144159840324
1.3867871347872511
1.386032212918428
1.385494414000199
1.3861752283209745
1.3859443846849846
1.3859489612255258
1.3870822239494038
1.386417973